In [ ]:
import requests

from bs4 import BeautifulSoup
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.chrome.options import Options
import pandas as pd

import time

In [ ]:
base_url = 'https://www.sephora.com'
crawl_delay=6


DRIVER_PATH = '../../chromedriver_mac64/chromedriver'

In [ ]:
options = Options()
options.headless = True
options.add_argument("--window-size=1920,1200")
options.add_argument('--disable-gpu')
options.add_argument('--no-sandbox')
options.add_experimental_option("excludeSwitches", ["enable-automation"])
options.add_experimental_option('useAutomationExtension', False)

user_agent = 'Mozilla/5.0 (X11; Linux x86_64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/60.0.3112.50 Safari/537.36'
options.add_argument('user-agent={0}'.format(user_agent))
driver = webdriver.Chrome(options=options, executable_path=DRIVER_PATH)
url = "https://www.sephora.com/ca/en/brands-list"
driver.get(url)
soup = BeautifulSoup(driver.page_source, 'html.parser')
driver.quit()

In [ ]:

# collecting brand names and links from brand list page 
brand_data = []
for brand_link in soup.findAll('a', attrs={"data-at": "brand_link"}):
    brand = {}
    brand['name'] = brand_link.span.text
    brand['link'] = brand_link.get('href') 
    brand_data.append(brand)


In [ ]:
# for each brand, grab products on brand page, save products as list of links 
for brand in brand_data:
    url = base_url+brand['link']
    driver = webdriver.Chrome(options=options, executable_path=DRIVER_PATH)
    driver.get(url)
    soup = BeautifulSoup(driver.page_source, 'html.parser')
    driver.quit()
    
    product_tiles = soup.find_all(attrs={"data-comp":"ProductTile "})
    brand['products'] = [product.get("href").split(" ")[0] for product in product_tiles]
    time.sleep(crawl_delay)
    break

# for each product page, determine how manys variations
# for example -> product/sol-de-janeiro-rio-radiance-perfume-mist - fullsize and mini
# how do you need to interact with the page to get volume etc for each



In [ ]:
# single product 
url = 'https://www.sephora.com/ca/en/product/charlotte-tilbury-airbrush-flawless-setting-spray-P461147?skuId=2368439&icid2=products%20grid:p461147:product'


driver = webdriver.Chrome(options=options, executable_path=DRIVER_PATH)
driver.get(url)
soup = BeautifulSoup(driver.page_source, 'html.parser')
driver.quit()

In [ ]:
print(soup.prettify())

In [ ]:
soup.find_all('data-comp')